In [1]:
######################################     Modules     #######################################
# MODULES
import sys
from time import process_time

import verbose_module
sys.path.append('../../.')
import pIRPgym


Set parameter Username
Academic license - for non-commercial use only - expires 2024-02-11


In [2]:
########################     Instance generator and Environment     #########################
### pIRP model's parameters
# Stochasticity
stochastic_params = ['d','q']
look_ahead = ['d','q']


# Historical data
historical_data = ['*']


# Other parameters
backorders = 'backorders'

# Random Instance
q_params = {'dist': 'c_uniform', 'r_f_params': [6,20]}          # Offer
p_params = {'dist': 'd_uniform', 'r_f_params': [20,61]}

d_params = {'dist': 'log-normal', 'r_f_params': [3,1]}          # Demand

h_params = {'dist': 'd_uniform', 'r_f_params': [20,61]}         # Holding costs

disc = ("strong","conc")


### Environment 
# Creating environment object
routing = True
inventory = True
perishability = 'ages'
env = pIRPgym.steroid_IRP(routing, inventory, perishability)

# Small instance (M = 7, K = 10)

In [3]:
################################## Policy Evaluation ##################################
env_config = {'M':7,'K':10,'T':12,'F':7,'Q':1500,
              'S':6,'LA_horizon':4,
             'd_max':2000,'hist_window':60,
             'back_o_cost':5000}

stoch_rd_seed = 0       # Random seeds
det_rd_seed = 0

# Creating instance generator object
inst_gen = pIRPgym.instance_generator(look_ahead, stochastic_params,
                              historical_data, backorders, env_config = env_config)

inst_gen.generate_basic_random_instance(det_rd_seed,stoch_rd_seed,q_params=q_params,
            p_params=p_params,d_params=d_params,h_params=h_params,discount=disc)

# Reseting the environment
state = env.reset(inst_gen,return_state=True)

''' Parameters '''
num_episodes = 1
verbose = True
start = process_time()

# for episode in num_episodes:

if verbose: string = verbose_module.print_iteration_head()
run_time = process_time()

# Reseting the environment
state = env.reset(inst_gen,return_state=True)

done = False
while not done:
    if verbose: string = verbose_module.print_step(env.t,start)

    ''' Purchase'''
    [purchase,demand_compliance], la_dec = pIRPgym.Inventory.Stochastic_Rolling_Horizon(state,env,inst_gen)
    if verbose:     string = verbose_module.print_purchase_update(string,inst_gen.W_p[env.t],purchase)

    ''' Routing '''
    nn_routes, nn_obj, nn_info, nn_time = pIRPgym.Routing.NearestNeighbor(purchase,inst_gen,env.t)                                         # Nearest Neighbor
    if verbose: string = verbose_module.print_routing_update(string,nn_obj,len(nn_routes))
    RCL_routes, RCL_obj, (RCL_distances, RCL_loads), RCL_time  = pIRPgym.Routing.RCL_Heuristic(purchase,inst_gen,env.t)                                 # RCL based constructive
    if verbose: string = verbose_module.print_routing_update(string,RCL_obj,len(RCL_routes))
    GA_routes,GA_obj,(GA_distances,GA_loads),GA_time,_ = pIRPgym.Routing.HybridGenticAlgorithm(purchase,inst_gen,env.t,return_top=False,rd_seed=0,time_limit=5)    # Genetic Algorithm
    if verbose: string = verbose_module.print_routing_update(string,sum(GA_distances),len(GA_routes))
    HyGeSe_routes, HyGeSe_distance, HyGeSe_time  = pIRPgym.Routing.HyGeSe.HyGeSe_routing(purchase,inst_gen,env.t,time_limit=5)                                   # Hybrid Genetic Search (CVRP)
    if verbose: string = verbose_module.print_routing_update(string,HyGeSe_distance,len(HyGeSe_routes))
    MIP_routes, MIP_obj, MIP_info, MIP_time = pIRPgym.Routing.MixedIntegerProgram(purchase,inst_gen,env.t)
    if verbose: string = verbose_module.print_routing_update(string,MIP_obj,len(MIP_info[0]))
    CG_routes, CG_distances, CG_loads, CG_time = pIRPgym.Routing.ColumnGeneration(purchase,inst_gen,env.t,verbose=False)       # Column Generation algorithm                  
    if verbose: string = verbose_module.print_routing_update(string,sum(CG_distances),len(CG_routes),end=True)
    
    pending_s,requirements = pIRPgym.Routing.consolidate_purchase(purchase,inst_gen,env.t)
    feasible,objective,(distances,loads) = pIRPgym.Routing_management.evaluate_routes(inst_gen,CG_routes,requirements)

    ''' Compound action'''        
    action = {'routing':CG_routes,'purchase':purchase,'demand_compliance':demand_compliance}

    state, reward, done, real_action, _,  = env.step(action,inst_gen)

print('Finished episode!!!')

**************************************************  pIRP environment  **************************************************
--------|-- Purchasing -|------------------------------------------- Routing -------------------------------------------
--------| Stochastic RH |       NN 	|       RCL 	|       HGA 	|       HGS*	|       MIP	|     CG
t t(s)	| cost 	real c.	| Obj 	 #Veh 	| Obj 	 #Veh 	| Obj 	 #Veh 	| Obj 	 #Veh 	| Obj 	 #Veh 	| Obj 	 #Veh 	
------------------------------------------------------------------------------------------------------------------------
1 0.0	|29572	29572	| 6457 	  4	| 5230 	  3	| 6457 	  4	| 3095 	  1	| 4831 	  3	| 4831 	  3	
2 12.38 |24112	24112	| 6457 	  4	| 4980 	  3	| 6457 	  4	| 3095 	  1	| 4831 	  3	| 4831 	  3	
3 24.02 |18699	18699	| 6311 	  4	| 6311 	  4	| 6311 	  4	| 3016 	  1	| 4815 	  3	| 4815 	  3	
4 32.78 |19689	19689	| 6457 	  4	| 5186 	  3	| 6457 	  4	| 3095 	  1	| 4831 	  3	| 4831 	  3	
5 46.45 |16402	16402	| 5021 	  3	| 5182 	  3	| 5021 	  3	| 

# Medium instances (M = 10, K = 14)

In [7]:
################################## Policy Evaluation ##################################
env_config = {'M':10,'K':14,'T':12,'F':10,'Q':2000,
              'S':6,'LA_horizon':4,
             'd_max':2000,'hist_window':60,
             'back_o_cost':5000}

stoch_rd_seed = 1       # Random seeds
det_rd_seed = 1

# Creating instance generator object
inst_gen = pIRPgym.instance_generator(look_ahead, stochastic_params,
                              historical_data,backorders,env_config = env_config)

inst_gen.generate_basic_random_instance(det_rd_seed,stoch_rd_seed,q_params=q_params,
            p_params=p_params,d_params=d_params,h_params=h_params,discount=disc)

# Reseting the environment
state = env.reset(inst_gen,return_state=True)

''' Parameters '''
num_episodes = 1
verbose = True
start = process_time()

# for episode in num_episodes:

if verbose: string = verbose_module.print_iteration_head()

# Episode's and performance storage
rewards=dict();  states=dict();   real_actions=dict();   backorders=dict();   la_decisions=dict()
perished=dict(); actions=dict(); #times=dict() 

routing_performance = dict()
run_time = process_time()

# Reseting the environment
state = env.reset(inst_gen,return_state=True)

done = False
while not done:
    if verbose: string = verbose_module.print_step(env.t,start)

    #print_state(env)
    # Environment transition
    states[env.t] = state 

    ''' Purchase'''
    [purchase,demand_compliance], la_dec = pIRPgym.Inventory.Stochastic_Rolling_Horizon(state,env,inst_gen)
    if verbose:     string = verbose_module.print_purchase_update(string,inst_gen.W_p[env.t],purchase)

    ''' Routing '''
    nn_routes, nn_obj, nn_info, nn_time = pIRPgym.Routing.NearestNeighbor(purchase,inst_gen,env.t)                                         # Nearest Neighbor
    if verbose: string = verbose_module.print_routing_update(string,nn_obj,len(nn_routes))
    RCL_routes, RCL_obj, (RCL_distances, RCL_loads), RCL_time  = pIRPgym.Routing.RCL_Heuristic(purchase,inst_gen,env.t)                                 # RCL based constructive
    if verbose: string = verbose_module.print_routing_update(string,RCL_obj,len(RCL_routes))
    GA_routes,GA_obj,(GA_distances,GA_loads),GA_time,_ = pIRPgym.Routing.HybridGenticAlgorithm(purchase,inst_gen,env.t,return_top=False,rd_seed=0,time_limit=30)    # Genetic Algorithm
    if verbose: string = verbose_module.print_routing_update(string,sum(GA_distances),len(GA_routes))
    HyGeSe_routes, HyGeSe_distance, HyGeSe_time  = pIRPgym.Routing.HyGeSe.HyGeSe_routing(purchase,inst_gen,env.t,time_limit=30)                                   # Hybrid Genetic Search (CVRP)
    if verbose: string = verbose_module.print_routing_update(string,HyGeSe_distance,len(HyGeSe_routes))
    MIP_routes, MIP_obj, MIP_info, MIP_time = pIRPgym.Routing.MixedIntegerProgram(purchase,inst_gen,env.t)
    if verbose: string = verbose_module.print_routing_update(string,MIP_obj,len(MIP_info[0]))
    CG_routes, CG_distances, CG_loads, CG_time = pIRPgym.Routing.ColumnGeneration(purchase,inst_gen,env.t,verbose=False)       # Column Generation algorithm                  
    if verbose: string = verbose_module.print_routing_update(string,sum(CG_distances),len(CG_routes),end=True)
    
    pending_s,requirements = pIRPgym.Routing.consolidate_purchase(purchase,inst_gen,env.t)
    feasible,objective,(distances,loads) = pIRPgym.Routing_management.evaluate_routes(inst_gen,CG_routes,requirements)

    ''' Compound action'''        
    action = {'routing':CG_routes,'purchase':purchase,'demand_compliance':demand_compliance}

    state, reward, done, real_action, _,  = env.step(action,inst_gen)
    if done:   states[env.t] = state
    
    # Data storage
    actions[env.t-1] = action
    real_actions[env.t-1] = real_action
    backorders[env.t-1] = _["backorders"]
    perished[env.t-1] = {k:_["perished"][k] if k in _["perished"] else 0 for k in inst_gen.Products}
    # rewards[env.t] = reward
    # la_decisions[env.t-1] = la_dec

print('Finished episode!!!')

**************************************************  pIRP environment  **************************************************
--------|-- Purchasing -|------------------------------------------- Routing -------------------------------------------
--------| Stochastic RH |       NN 	|       RCL 	|       HGA 	|       HGS*	|       MIP	|     CG
t t(s)	| cost 	real c.	| Obj 	 #Veh 	| Obj 	 #Veh 	| Obj 	 #Veh 	| Obj 	 #Veh 	| Obj 	 #Veh 	| Obj 	 #Veh 	
------------------------------------------------------------------------------------------------------------------------
1 0.0	|35335	35335	| 6576 	  4	| 6866 	  4	| 6576 	  4	| 3146 	  1	| 5291 	  3	| 5291 	  3	
2 316.7 |27685	27685	| 4868 	  3	| 5357 	  3	| 4868 	  3	| 2793 	  1	| 4430 	  3	| 4430 	  3	
3 439.1 |25640	25640	| 6575 	  4	| 6859 	  4	| 6575 	  4	| 3085 	  1	| 5266 	  3	| 5266 	  3	
4 525.8 |19440	19440	| 6572 	  4	| 6856 	  4	| 6572 	  4	| 3137 	  1	| 5255 	  3	| 5255 	  3	
5 633.2 |20231	20231	| 6575 	  4	| 6635 	  4	| 6575 	  4	| 

# Large instances

## M = 20, K = 20

In [3]:
env_config = {'M':20,'K':20,'T':8,'F':20,'Q':1500,
              'S':6,'LA_horizon':4,
             'd_max':2000,'hist_window':60,
             'back_o_cost':5000}

stoch_rd_seed = 2       # Random seeds
det_rd_seed = 2

# Creating instance generator object
inst_gen = pIRPgym.instance_generator(look_ahead, stochastic_params,
                              historical_data,backorders,env_config = env_config)

inst_gen.generate_basic_random_instance(det_rd_seed,stoch_rd_seed,q_params=q_params,
            p_params=p_params,d_params=d_params,h_params=h_params,discount=disc)

# Reseting the environment
state = env.reset(inst_gen,return_state=True)

''' Parameters '''
num_episodes = 1
verbose = True
start = process_time()

# for episode in num_episodes:

if verbose: string = verbose_module.print_iteration_head()

# Episode's and performance storage
rewards=dict();  states=dict();   real_actions=dict();   backorders=dict();   la_decisions=dict()
perished=dict(); actions=dict(); #times=dict() 

routing_performance = dict()
run_time = process_time()

# Reseting the environment
state = env.reset(inst_gen,return_state=True)

done = False
while not done:
    if verbose: string = verbose_module.print_step(env.t,start)

    #print_state(env)
    # Environment transition
    states[env.t] = state 

    ''' Purchase'''
    [purchase,demand_compliance], la_dec = pIRPgym.Inventory.Stochastic_Rolling_Horizon(state,env,inst_gen)
    if verbose:     string = verbose_module.print_purchase_update(string,inst_gen.W_p[env.t],purchase)

    ''' Routing '''
    nn_routes, nn_obj, nn_info, nn_time = pIRPgym.Routing.NearestNeighbor(purchase,inst_gen,env.t)                                         # Nearest Neighbor
    if verbose: string = verbose_module.print_routing_update(string,nn_obj,len(nn_routes))
    RCL_routes, RCL_obj, (RCL_distances, RCL_loads), RCL_time  = pIRPgym.Routing.RCL_Heuristic(purchase,inst_gen,env.t)                                 # RCL based constructive
    if verbose: string = verbose_module.print_routing_update(string,RCL_obj,len(RCL_routes))
    GA_routes,GA_obj,(GA_distances,GA_loads),GA_time,_ = pIRPgym.Routing.HybridGenticAlgorithm(purchase,inst_gen,env.t,return_top=False,rd_seed=0,time_limit=30)    # Genetic Algorithm
    if verbose: string = verbose_module.print_routing_update(string,sum(GA_distances),len(GA_routes))
    HyGeSe_routes, HyGeSe_distance, HyGeSe_time  = pIRPgym.Routing.HyGeSe.HyGeSe_routing(purchase,inst_gen,env.t,time_limit=30)                                   # Hybrid Genetic Search (CVRP)
    if verbose: string = verbose_module.print_routing_update(string,HyGeSe_distance,len(HyGeSe_routes))
    # MIP_routes, MIP_obj, MIP_info, MIP_time = pIRPgym.Routing.MixedIntegerProgram(purchase,inst_gen,env.t)
    # if verbose: string = verbose_module.print_routing_update(string,MIP_obj,len(MIP_info[0]))
    CG_routes, CG_distances, CG_loads, CG_time = pIRPgym.Routing.ColumnGeneration(purchase,inst_gen,env.t,verbose=False)       # Column Generation algorithm                  
    if verbose: string = verbose_module.print_routing_update(string,sum(CG_distances),len(CG_routes),end=True)
    
    pending_s,requirements = pIRPgym.Routing.consolidate_purchase(purchase,inst_gen,env.t)
    feasible,objective,(distances,loads) = pIRPgym.Routing_management.evaluate_routes(inst_gen,CG_routes,requirements)

    ''' Compound action'''        
    action = {'routing':CG_routes,'purchase':purchase,'demand_compliance':demand_compliance}

    state, reward, done, real_action, _,  = env.step(action,inst_gen)
    if done:   states[env.t] = state
    
    # Data storage
    actions[env.t-1] = action
    real_actions[env.t-1] = real_action
    backorders[env.t-1] = _["backorders"]
    perished[env.t-1] = {k:_["perished"][k] if k in _["perished"] else 0 for k in inst_gen.Products}
    # rewards[env.t] = reward
    # la_decisions[env.t-1] = la_dec

print('Finished episode!!!')

**************************************************  pIRP environment  **************************************************
--------|-- Purchasing -|------------------------------------------- Routing -------------------------------------------
--------| Stochastic RH |       NN 	|       RCL 	|       HGA 	|       HGS*	|       MIP	|     CG
t t(s)	| cost 	real c.	| Obj 	 #Veh 	| Obj 	 #Veh 	| Obj 	 #Veh 	| Obj 	 #Veh 	| Obj 	 #Veh 	| Obj 	 #Veh 	
------------------------------------------------------------------------------------------------------------------------
1 0.0	|69453	69453	| 7139 	  4	| 6319 	  4	| 7139 	  4	| 4450 	  2	| 5342 	  3	
2 763.1 |40255	40255	| 7046 	  4	| 7189 	  4	| 7046 	  4	| 3837 	  1	| 4990 	  3	
3 998.3 |40004	40004	| 5980 	  4	| 6360 	  4	| 5980 	  4	| 3666 	  1	| 4741 	  3	
4 1077.9 |20317	20317	| 3414 	  2	| 3679 	  2	| 3414 	  2	| 2817 	  1	| 2988 	  2	
5 1121.6 |27518	27518	| 5415 	  3	| 6017 	  4	| 5415 	  3	| 3743 	  1	| 4693 	  3	
6 1173.3 |28033	28033	|

## M = 25, K = 28

In [3]:
env_config = {'M':25,'K':28,'T':8,'F':25,'Q':2000,
              'S':6,'LA_horizon':4,
             'd_max':2000,'hist_window':60,
             'back_o_cost':5000}

stoch_rd_seed = 3       # Random seeds
det_rd_seed = 33

# Creating instance generator object
inst_gen = pIRPgym.instance_generator(look_ahead, stochastic_params,
                              historical_data,backorders,env_config = env_config)

inst_gen.generate_basic_random_instance(det_rd_seed,stoch_rd_seed,q_params=q_params,
            p_params=p_params,d_params=d_params,h_params=h_params,discount=disc)

# Reseting the environment
state = env.reset(inst_gen,return_state=True)

''' Parameters '''
num_episodes = 1
verbose = True
start = process_time()

# for episode in num_episodes:

if verbose: string = verbose_module.print_iteration_head()

# Episode's and performance storage
rewards=dict();  states=dict();   real_actions=dict();   backorders=dict();   la_decisions=dict()
perished=dict(); actions=dict(); #times=dict() 

routing_performance = dict()
run_time = process_time()

# Reseting the environment
state = env.reset(inst_gen,return_state=True)

done = False
while not done:
    if verbose: string = verbose_module.print_step(env.t,start)

    #print_state(env)
    # Environment transition
    states[env.t] = state 

    ''' Purchase'''
    [purchase,demand_compliance], la_dec = pIRPgym.Inventory.Stochastic_Rolling_Horizon(state,env,inst_gen)
    if verbose:     string = verbose_module.print_purchase_update(string,inst_gen.W_p[env.t],purchase)

    ''' Routing '''
    nn_routes, nn_obj, nn_info, nn_time = pIRPgym.Routing.NearestNeighbor(purchase,inst_gen,env.t)                                         # Nearest Neighbor
    if verbose: string = verbose_module.print_routing_update(string,nn_obj,len(nn_routes))
    RCL_routes, RCL_obj, (RCL_distances, RCL_loads), RCL_time  = pIRPgym.Routing.RCL_Heuristic(purchase,inst_gen,env.t)                                 # RCL based constructive
    if verbose: string = verbose_module.print_routing_update(string,RCL_obj,len(RCL_routes))
    GA_routes,GA_obj,(GA_distances,GA_loads),GA_time,_ = pIRPgym.Routing.HybridGenticAlgorithm(purchase,inst_gen,env.t,return_top=False,rd_seed=0,time_limit=30)    # Genetic Algorithm
    if verbose: string = verbose_module.print_routing_update(string,sum(GA_distances),len(GA_routes))
    HyGeSe_routes, HyGeSe_distance, HyGeSe_time  = pIRPgym.Routing.HyGeSe.HyGeSe_routing(purchase,inst_gen,env.t,time_limit=30)                                   # Hybrid Genetic Search (CVRP)
    if verbose: string = verbose_module.print_routing_update(string,HyGeSe_distance,len(HyGeSe_routes))
    # MIP_routes, MIP_obj, MIP_info, MIP_time = pIRPgym.Routing.MixedIntegerProgram(purchase,inst_gen,env.t)
    # if verbose: string = verbose_module.print_routing_update(string,MIP_obj,len(MIP_info[0]))
    CG_routes, CG_distances, CG_loads, CG_time = pIRPgym.Routing.ColumnGeneration(purchase,inst_gen,env.t,verbose=False)       # Column Generation algorithm                  
    if verbose: string = verbose_module.print_routing_update(string,sum(CG_distances),len(CG_routes),end=True)
    
    pending_s,requirements = pIRPgym.Routing.consolidate_purchase(purchase,inst_gen,env.t)
    feasible,objective,(distances,loads) = pIRPgym.Routing_management.evaluate_routes(inst_gen,CG_routes,requirements)

    ''' Compound action'''        
    action = {'routing':CG_routes,'purchase':purchase,'demand_compliance':demand_compliance}

    state, reward, done, real_action, _,  = env.step(action,inst_gen)
    if done:   states[env.t] = state
    
    # Data storage
    actions[env.t-1] = action
    real_actions[env.t-1] = real_action
    backorders[env.t-1] = _["backorders"]
    perished[env.t-1] = {k:_["perished"][k] if k in _["perished"] else 0 for k in inst_gen.Products}
    # rewards[env.t] = reward
    # la_decisions[env.t-1] = la_dec

print('Finished episode!!!')

**************************************************  pIRP environment  **************************************************
--------|-- Purchasing -|------------------------------------------- Routing -------------------------------------------
--------| Stochastic RH |       NN 	|       RCL 	|       HGA 	|       HGS*	|       MIP	|     CG
t t(s)	| cost 	real c.	| Obj 	 #Veh 	| Obj 	 #Veh 	| Obj 	 #Veh 	| Obj 	 #Veh 	| Obj 	 #Veh 	| Obj 	 #Veh 	
------------------------------------------------------------------------------------------------------------------------
1 0.0	|100599	100599	| 8841 	  5	| 9450 	  5	| 8841 	  5	| 5040 	  2	|
Interrupt request received
1 0.0	|100599	100599	| 8841 	  5	| 9450 	  5	| 8841 	  5	| 5040 	  2	| 7266 	  4	


## M = 50, K = 20

In [5]:
env_config = {'M':50,'K':20,'T':8,'F':50,'Q':1500,
              'S':6,'LA_horizon':4,
             'd_max':2000,'hist_window':60,
             'back_o_cost':5000}

stoch_rd_seed = 4       # Random seeds
det_rd_seed = 34

# Creating instance generator object
inst_gen = pIRPgym.instance_generator(look_ahead, stochastic_params,
                              historical_data,backorders,env_config = env_config)

inst_gen.generate_basic_random_instance(det_rd_seed,stoch_rd_seed,q_params=q_params,
            p_params=p_params,d_params=d_params,h_params=h_params,discount=disc)

# Reseting the environment
state = env.reset(inst_gen,return_state=True)

''' Parameters '''
num_episodes = 1
verbose = True
start = process_time()

# for episode in num_episodes:

if verbose: string = verbose_module.complete_progress.print_iteration_head(exclude_MIP=True)

# Episode's and performance storage
rewards=dict();  states=dict();   real_actions=dict();   backorders=dict();   la_decisions=dict()
perished=dict(); actions=dict(); #times=dict() 

routing_performance = dict()
run_time = process_time()

# Reseting the environment
state = env.reset(inst_gen,return_state=True)

done = False
while not done:
    if verbose: string = verbose_module.complete_progress.print_step(env.t,start)

    #print_state(env)
    # Environment transition
    states[env.t] = state 

    ''' Purchase'''
    [purchase,demand_compliance], la_dec = pIRPgym.Inventory.Stochastic_Rolling_Horizon(state,env,inst_gen)
    if verbose:     string = verbose_module.complete_progress.print_purchase_update(string,inst_gen.W_p[env.t],purchase)

    ''' Routing '''
    nn_routes, nn_obj, nn_info, nn_time = pIRPgym.Routing.NearestNeighbor(purchase,inst_gen,env.t)                                         # Nearest Neighbor
    if verbose: string = verbose_module.complete_progress.print_routing_update(string,nn_obj,len(nn_routes))
    RCL_routes, RCL_obj, (RCL_distances, RCL_loads), RCL_time  = pIRPgym.Routing.RCL_Heuristic(purchase,inst_gen,env.t)                                 # RCL based constructive
    if verbose: string = verbose_module.complete_progress.print_routing_update(string,RCL_obj,len(RCL_routes))
    GA_routes,GA_obj,(GA_distances,GA_loads),GA_time,_ = pIRPgym.Routing.HybridGenticAlgorithm(purchase,inst_gen,env.t,return_top=False,rd_seed=0,time_limit=30)    # Genetic Algorithm
    if verbose: string = verbose_module.complete_progress.print_routing_update(string,sum(GA_distances),len(GA_routes))
    HyGeSe_routes, HyGeSe_distance, HyGeSe_time  = pIRPgym.Routing.HyGeSe.HyGeSe_routing(purchase,inst_gen,env.t,time_limit=30)                                   # Hybrid Genetic Search (CVRP)
    if verbose: string = verbose_module.complete_progress.print_routing_update(string,HyGeSe_distance,len(HyGeSe_routes))
    # MIP_routes, MIP_obj, MIP_info, MIP_time = pIRPgym.Routing.MixedIntegerProgram(purchase,inst_gen,env.t)
    # if verbose: string = verbose_module.complete_progress.print_routing_update(string,MIP_obj,len(MIP_info[0]))
    CG_routes, CG_distances, CG_loads, CG_time = pIRPgym.Routing.ColumnGeneration(purchase,inst_gen,env.t,verbose=False)       # Column Generation algorithm                  
    if verbose: string = verbose_module.complete_progress.print_routing_update(string,sum(CG_distances),len(CG_routes),end=True,CG_time=CG_time)
    
    pending_s,requirements = pIRPgym.Routing.consolidate_purchase(purchase,inst_gen,env.t)
    feasible,objective,(distances,loads) = pIRPgym.Routing_management.evaluate_routes(inst_gen,CG_routes,requirements)

    ''' Compound action'''        
    action = {'routing':CG_routes,'purchase':purchase,'demand_compliance':demand_compliance}

    state, reward, done, real_action, _,  = env.step(action,inst_gen)
    if done:   states[env.t] = state
    
    # Data storage
    actions[env.t-1] = action
    real_actions[env.t-1] = real_action
    backorders[env.t-1] = _["backorders"]
    perished[env.t-1] = {k:_["perished"][k] if k in _["perished"] else 0 for k in inst_gen.Products}
    # rewards[env.t] = reward
    # la_decisions[env.t-1] = la_dec

print('Finished episode!!!')

**********************************************  pIRP environment  **********************************************
--------|-- Purchasing -|--------------------------------------- Routing ---------------------------------------
--------| Stochastic RH |       NN 	|       RCL 	|       HGA 	|       HGS*	| 	   CG	 	|
t t(s)	| cost 	real c.	| #Veh 	 Obj 	| #Veh 	 Obj 	| #Veh 	 Obj 	| #Veh 	 Obj 	| t(s)	  #Veh 	 Obj 	| 
----------------------------------------------------------------------------------------------------------------
1 0.0	|50468	50468	| 5907 	  3	| 6198 	  4	| 5907 	  3	| 3715 	  2	|
Interrupt request received
1 0.0	|50468	50468	| 5907 	  3	| 6198 	  4	| 5907 	  3	| 3715 	  2	| 335.3	4689 	  3	|
